# Import & Load NESI data

### Config

In [1]:
import configparser
config = configparser.ConfigParser()
config.read_file(open('../../settings.ini'))

engine_path = config.get('DATABASE','engine_path')

### Main params

In [2]:
remote_path = 'http://pacha.datawheel.us/economia/nesi/nesi_all_years/'
local_path = '../data/'

### Imports

In [3]:
from urllib import request
import zipfile
import shutil
import os.path

import json
import pandas as pd
from sqlalchemy import create_engine

### Open file function

In [4]:
def loadFile(file_name):
    remote_file = remote_path + file_name
    local_file = local_path + file_name

    if not os.path.isfile(local_file):
        with request.urlopen(remote_file) as remote_csv,open(local_file, 'wb') as local_csv:
            shutil.copyfileobj(remote_csv, local_csv)
    
    return pd.read_csv(local_file,delimiter=",")

### Load & rename

In [5]:
df = loadFile('nesi_all_years.csv')
list(df)

['year',
 'comuna_datachile_id',
 'region_id',
 'income',
 'fact',
 'sex_id',
 'journey_id',
 'isco_id',
 'icse_id',
 'isced_id']

### Ingest

In [6]:
df = df.astype({'year':'int','comuna_datachile_id':'int','region_id':'int','income':'int','sex_id':'int','journey_id':'int','isco_id':'int','icse_id':'int','isced_id':'int'})

engine = create_engine(engine_path)
df.to_sql('fact_income_nesi', engine, schema='economy', if_exists='replace', index=False)

In [7]:
### Foreign & indexes

In [8]:
engine.execute("""
ALTER TABLE economy.fact_income_nesi
  ADD COLUMN date_id INTEGER; 
""")

engine.execute("""
UPDATE economy.fact_income_nesi
SET date_id = dim_date.id
FROM public.dim_date
WHERE dim_date.the_year = economy.fact_income_nesi.year
      AND dim_date.month_of_year = 1
      AND dim_date.day_of_month = 1
""")

engine.execute("""
CREATE INDEX fact_income_nesi_region_id 
ON economy.fact_income_nesi (region_id)
""")

engine.execute("""
CREATE INDEX fact_income_nesi_comuna_datachile_id
ON economy.fact_income_nesi (comuna_datachile_id)
""")

### Related dim

In [9]:
remote_path = 'http://pacha.datawheel.us/ids_oficiales/'
d1 = loadFile('cine_isced.csv')
d1

,isced,isced_id
0,Doctorado,1
1,Educación Preescolar,2
2,Educación Primaria (nivel 1),3
3,Educación Primaria (nivel 2),4
4,Educación Secundaria,5
5,Educación Técnica (Educación Superior no Unive...,6
6,Educación Universitaria,7
7,Nivel ignorado,8
8,Nunca estudió,9
9,Postítulo y Maestría,10


In [10]:
d2 = loadFile('cise_icse.csv')
d2

,icse,icse_id
0,Asalariado Sector Privado,1
1,Asalariado Sector Público,2
2,Cuenta Propia,3
3,Empleador,4
4,Personal de Servicio Doméstico Puertas Adentro,5
5,Personal de Servicio Doméstico Puertas Afuera,6
6,No sabe/No responde,0


In [11]:
d3 = loadFile('ciuo_isco.csv')
d3

,isco,isco_id
0,Agricultores y trabajadores calificados agrope...,1
1,Empleados de oficina,2
2,Funcionarios públicos y personal directivo de ...,3
3,"Oficiales, operarios y artesanos de artes mecá...",4
4,Operadores de instalaciones y máquinas y monta...,5
5,Otros no identificados,6
6,Profesionales científicos e intelectuales,7
7,Técnicos y profesionales de nivel medio,8
8,Trabajadores de los servicios y vendedores de ...,9
9,Trabajadores no calificados,10


In [12]:
d4 = loadFile('sex.csv')
d4

,sex_id,sex
0,0,Hombre
1,1,Mujer


In [13]:
d5 = loadFile('journey.csv')
d5

,journey_id,journey
0,0,Parcial
1,1,Completa
